# Accumulate and save SID uncertainty
$\sigma_{i} = \sqrt{<\sigma_{i-1}^2> + \sigma_S^2}$

$\sigma_S$ - uncertainty of smoothed SID from CDR

$\sigma_{i-1}$ - uncertainty from previous step

$<>$ - advection operation

$\sigma_S = \frac{\sqrt{\sum_j^N{\sigma_{Oj}}}}{\sqrt{N}}$

$\sigma_{Oj}$ - uncertaimty in neighbour $j$

$N$ - number of neighbours
"""

In [ ]:
import os
from datetime import datetime, timedelta
import glob
from multiprocessing import Pool

import numpy as np

from lmsiage.uncertainty import ComputeSidUncertainty
from lmsiage.mesh_file import MeshFile

from lmsiage.zarr_index_tools import (update_index_for_dir, 
                                      files_missing_array, 
                                      delete_file_from_index, 
                                      files_with_array, 
                                      cleanup_missing_files)
#update_index_for_dir(unc_dir)
#files_missing_array('unc_sid')
cleanup_missing_files()

In [ ]:
# filter size (Gaissian filter was 5, but weights are smaller for larger distances)
size = 3

# should be 2210 to cover 6 years
n_steps = 2210 

mesh_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh'
unc_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert'

sid_dir = 'OSISAF_ice_drift_CDR_postproc'
sid_files = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
sid_dates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in sid_files]
print(f'Found {len(sid_files)} ice drift files from {sid_dates[0]} to {sid_dates[-1]}')

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']

In [ ]:
csidu = ComputeSidUncertainty(mesh_dir, unc_dir, sid_files, sid_dates, n_steps, size, xc, yc)
for year in [1991, 1992, 1993, 1994]:
    csidu.compute_smearing_uncertainty(year)

In [ ]:
unc_sid_files = sorted(files_with_array('unc_sid'))
unc_sid_dates = [datetime.strptime(f.split('/')[-1], 'unc_%Y%m%d.zip') for f in unc_sid_files]
start_indices = [i for i in range(0, len(unc_sid_dates)) if unc_sid_dates[i].month == 9 and unc_sid_dates[i].day == 15]
for start_idx in start_indices:
    csidu.compute_integrated_uncertainty(unc_sid_files, unc_sid_dates, start_idx)

In [ ]:
MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/1994/unc_19941231.zip').read_names()

In [ ]:
import matplotlib.pyplot as plt
unc_sid = MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/1994/unc_19941231.zip').load(['unc_sid1991'], as_dict=False)[0]
x, y, t = MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/1994/mesh_19941231.zip').load(['x', 'y', 't'], as_dict=False)
plt.tripcolor(x, y, t, unc_sid)
plt.colorbar()